In [1]:
from stop_words import get_stop_words

stop_words = get_stop_words('en')
stop_words[:10]

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and']

In [4]:
import os
rootDataPath = "data/texts"
for languageDirName in os.listdir(rootDataPath):
    languageDataPath = os.path.join(rootDataPath, languageDirName)
    for textFile in os.listdir(languageDataPath):
        print(languageDirName + textFile)

DutchDe kennemer vrijbuiter - Cornelis Johannes Kieviet.txt
DutchBerend Veltink - Harm Boom.txt
DutchBroeder en Zuster - Cyriel Buysse.txt
Dutchdesktop.ini
DutchBaas Gansendonck - Hendrik Conscience.txt
DutchDe baanwachter - Hendrik Conscience.txt
GermanOchsenkrieg - Ludwig Ganghofer.txt
GermanDer Waldlaufer - Gabriel Ferry.txt
GermanJenny - Fanny Lewald.txt
GermanDer Sandmann - Ernst Theodor Amadeus Hoffmann.txt
GermanDer Tee der drei alten Damen - Friedrich Glauser.txt
GermanReisen in die Felsengebirge Nordamerikas - Balduin Mollhausen.txt
GermanMichael Kohlhaas - Heinrich von Kleist.txt
GermanDer Sternsteinhof - Ludwig Anzengruber.txt
GermanDie Ahnen - Gustav Freytag.txt
GermanEin Kampf um Rom - Felix Dahn.txt
GermanDas Buch der Bilder - Rainer Maria Rilke.txt
GermanSudamerika - Friedrich Gerstacker.txt
GermanNussknacker und Mausekonig - Ernst Theodor Amadeus Hoffmann.txt
GermanDas Naturforscherschiff - Sophie Worishoffer.txt
GermanDer Prozess - Franz Kafka.txt
GermanDeutschland. Ei

In [5]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark Count words") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [44]:
stopwordLocalMappings = {
    "German" : "de",
    "English" : "en"
}

In [140]:
file = spark.sparkContext.wholeTextFiles("data/test/*/*.txt")
filesByLang = file.map(lambda x: (x[0].split("/")[-2], x[1])).reduceByKey(lambda v1,v2: v1+ " " + v2)
seqOp = (lambda x, y: (x[0], y[0]))
combOp = (lambda x, y:(x, y))
occurencesPerLanguage = filesByLang.flatMapValues(lambda l: re.findall("\w+", l))\
        .map(lambda x: (x[0], x[1].lower()))\
        .filter(lambda x: (x[1] not in get_stop_words(stopwordLocalMappings[x[0]])))\
        .map(lambda w: ((w[0], w[1]), 1)).reduceByKey(lambda v1,v2: v1+v2)\
        .map(lambda x: ((x[0][0]), x[0][1], x[1]))\
        .sortBy(lambda a: a[2], ascending=False)\
        .map(lambda x: (x[0], x[1]))
        #.groupBy(lambda x: x[0])


occurencesPerLanguage.collect()

[('German', 'test'),
 ('English', 'hey'),
 ('German', 'hallo'),
 ('English', 'test'),
 ('English', 'never'),
 ('English', 'know'),
 ('German', 'neuer'),
 ('German', 'tolle')]

[('English', 'hey'),
 ('English', 'test'),
 ('English', 'never'),
 ('English', 'know'),
 ('German', 'hallo'),
 ('German', 'test'),
 ('German', 'test'),
 ('German', 'neuer'),
 ('German', 'test')]

In [45]:
import re
file = spark.sparkContext.textFile("data/test/*/*.txt")
counts = file.flatMap(lambda l: re.findall("\w+", l))\
            .map(lambda x: x.lower())\
            .filter(lambda x: x not in get_stop_words('de'))\
            .map(lambda w: (w.lower(), 1)).reduceByKey(lambda v1,v2: v1+v2)\
            .sortBy(lambda a: a[1], ascending=False)

outputs = counts.collect()

In [46]:
outputs[:10]

[('test', 3),
 ('know', 1),
 ('is', 1),
 ('because', 1),
 ('you', 1),
 ('hey', 1),
 ('this', 1),
 ('a', 1),
 ('never', 1),
 ('hallo', 1)]

In [27]:
"die" in get_stop_words('de')

True

In [11]:
# Count words
words = spark.sparkContext.textFile("data/texts/German/*.txt").flatMap(lambda line: line.split(" "))
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)
outputs = wordCounts.collect()

In [12]:
outputs[:10]

[('', 121303),
 ('Bis', 234),
 ('aus', 14743),
 ('Geruchsklavier', 4),
 ('die', 131080),
 ('praktisch,', 6),
 ('unartige', 3),
 ('an', 24002),
 ('ein', 35588),
 ('vergebens', 156)]

In [50]:
spark.sparkContext.wholeTextFiles('data/texts/German/').map(lambda x:(x[0].split("texts/")[1].split("/")[0], x[1].split(" "))).collect()[0]





('German',
 ['Ludwig',
  'Ganghofer\r\n\r\nOchsenkrieg\r\n\r\n\r\n\r\n\r\n\r\nErstes',
  'Buch\r\n\r\n\r\n\r\n1\r\n\r\n\r\n\r\nEr',
  'nahm',
  'den',
  'Zügel',
  'straffer',
  'und',
  'versetzte',
  'dem',
  'schnaubenden',
  'Gaul',
  'einen',
  'Hieb',
  'mit',
  'der',
  'Reitpeitsche.',
  'Das',
  'Tier',
  'zuckte',
  'zusammen,',
  'bewegte',
  'aber',
  'keinen',
  'Huf,',
  'stierte',
  'mit',
  'vorgequollenen',
  'Augen',
  'auf',
  'die',
  'grauen',
  'Tümpel',
  'des',
  'Sumpfbodens',
  'und',
  'fing',
  'zu',
  'zittern',
  'an.\r\n\r\nDem',
  'jungen',
  'Reiter',
  'brannte',
  'der',
  'Zorn',
  'im',
  'Gesicht.',
  'Wieder',
  'hob',
  'er',
  'die',
  'Peitsche.',
  'Doch',
  'er',
  'schlug',
  'nicht,',
  'ließ',
  'die',
  'Gerte',
  'sinken',
  'und',
  'schob',
  'sie',
  'hinter',
  'den',
  'Ledergürtel.',
  'Während',
  'er',
  'unter',
  'beruhigendem',
  'Zureden',
  'dem',
  'Pferde',
  'den',
  'Hals',
  'tätschelte,',
  'sah',
  'er',
  'prüfend',
